In [1]:
import pandas as pd
import os

# 현재 스크립트 위치를 기준으로 경로 설정
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))  # analysis 폴더의 상위 폴더로 이동
data_dir = os.path.join(base_dir, 'data')  # data 폴더 경로
processed_dir = os.path.join(data_dir, 'processed')  # processed 폴더 경로

# 파일 경로 설정
climate_data_path = os.path.join(processed_dir, 'preprocessed_climate.csv')
market_data_path = os.path.join(processed_dir, 'preprocessed_agromarket.csv')

# 데이터 불러오기
dfc = pd.read_csv(climate_data_path)
dfm = pd.read_csv(market_data_path)

In [2]:
# 날짜 컬럼 변환
dfc['일시'] = pd.to_datetime(dfc['일시'], format='%Y-%m-%d')
dfm['일자'] = pd.to_datetime(dfm['일자'].astype(str), format='%Y%m%d')

In [3]:
# dfm에서 2024-12-20 데이터 제거
dfm = dfm[dfm['일자'] <= '2024-12-19']

In [4]:
# 날씨 데이터 결측치 데이터 생성
# 1) 날짜 범위 생성
full_date_range = pd.date_range(start=dfc['일시'].min(), end=dfc['일시'].max())  # 전체 날짜 범위 생성

# 2) 각 지점별로 데이터 재구성 및 채우기
df_filled = (
    dfc.set_index('일시')  # '일시'를 인덱스로 설정
    .groupby('지점명', group_keys=False)  # 지점별로 그룹화
    .apply(lambda group: group.reindex(full_date_range).ffill())  # 전체 날짜 범위로 재구성 후 이전 값으로 채움
)

# 3) 지점명 복구
df_filled['지점명'] = df_filled['지점명'].ffill()

# 4) 인덱스를 복원하여 '일시' 컬럼으로 변환
df_filled = df_filled.reset_index().rename(columns={'index': '일시'})

# 5) 결과 확인 (군산 2022-10-05 결측치)
print(df_filled[(df_filled['지점명'] == '군산') & 
                (df_filled['일시'] >= '2022-10-04') & 
                (df_filled['일시'] <= '2022-10-06')]) 

# 6) 원본 데이터 대체
dfc = df_filled.copy()

             일시 지점명  평균기온(°C)  최저기온(°C)  최고기온(°C)  일강수량(mm)  최대 풍속(m/s)  \
7904 2022-10-04  군산      23.9      20.1      24.6      13.6         7.4   
7905 2022-10-05  군산      23.9      20.1      24.6      13.6         7.4   
7906 2022-10-06  군산      23.9      15.5      20.9      13.6         5.6   

      평균 풍속(m/s)  평균 상대습도(%)  평균 현지기압(hPa)  합계 일조시간(hr)  일 최심신적설(cm)  
7904         5.0        86.9        1011.0          0.0          NaN  
7905         5.0        86.9        1011.0          0.0          NaN  
7906         3.3        86.9        1011.0          0.5          NaN  


C:\Users\skfkz\AppData\Local\Temp\ipykernel_7592\178767433.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.reindex(full_date_range).ffill())  # 전체 날짜 범위로 재구성 후 이전 값으로 채움


In [5]:
# 도매시장의 지역 날씨를 위한 맵핑

#도매시장 지역명을 앞 2글자로 변환
dfm['region'] = dfm['도매시장'].str[:2]

special_map = {
    '익산': '군산',
    '구리': '서울',
    '안양': '수원',
    '안산': '인천'
}

dfm['region'] = dfm['region'].replace(special_map)

In [6]:
#도매시장 데이터와 날씨 데이터 조인
dfc = dfc.set_index(['지점명', '일시'])

df_merged = pd.merge(
    dfm,
    dfc,
    left_on=['region', '일자'],
    right_index=True,
    how='left'
)

df_merged = df_merged.reset_index(drop=True)

In [7]:
#기후 관련 결측값(NaN)을 0으로 대체
weather_cols = [
    '평균기온(°C)', '최저기온(°C)', '최고기온(°C)',
    '일강수량(mm)', '최대 풍속(m/s)', '평균 풍속(m/s)',
    '평균 상대습도(%)', '평균 현지기압(hPa)', '합계 일조시간(hr)', '일 최심신적설(cm)'
]
for col in weather_cols:
    if col in df_merged.columns:
        df_merged[col] = df_merged[col].fillna(0)

In [8]:
# 조인된 데이터 csv로 저장
joined_data_path = os.path.join(processed_dir, 'joined_market_climate.csv')
df_merged.to_csv(joined_data_path, index=False)

In [9]:
# 기본 정보 확인
print("=== 결합된 df_merged 기본 정보 ===")
print(df_merged.info())
print(df_merged.head())

=== 결합된 df_merged 기본 정보 ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6387444 entries, 0 to 6387443
Data columns (total 18 columns):
 #   Column        Dtype         
---  ------        -----         
 0   일자            datetime64[ns]
 1   부류            object        
 2   품목            object        
 3   품종            object        
 4   도매시장          object        
 5   물량(kg)        float64       
 6   금액(원)         float64       
 7   region        object        
 8   평균기온(°C)      float64       
 9   최저기온(°C)      float64       
 10  최고기온(°C)      float64       
 11  일강수량(mm)      float64       
 12  최대 풍속(m/s)    float64       
 13  평균 풍속(m/s)    float64       
 14  평균 상대습도(%)    float64       
 15  평균 현지기압(hPa)  float64       
 16  합계 일조시간(hr)   float64       
 17  일 최심신적설(cm)   float64       
dtypes: datetime64[ns](1), float64(12), object(5)
memory usage: 877.2+ MB
None
          일자     부류   품목         품종 도매시장  물량(kg)      금액(원) region  평균기온(°C)  \
0 2019-02-01    버섯류 